In [1]:
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import AutoModelForCausalLM, AutoTokenizer
from datasets import load_dataset
from tqdm import tqdm
from torch.optim import Adam

In [2]:
raw_dataset = load_dataset("Dahoas/synthetic-instruct-gptj-pairwise",split="train[:10%]")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


dataset_infos.json: 0.00B [00:00, ?B/s]

(…)-00000-of-00001-1e5d57b93c448e7a.parquet:   0%|          | 0.00/18.2M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/33143 [00:00<?, ? examples/s]

In [4]:
dataset = DPODataset(raw_dataset)

In [3]:
class DPODataset(Dataset):
  def __init__(self,hf_dataset):
    self.dataset = hf_dataset

  def __len__(self):
    return len(self.dataset)

  def __getitem__(self,idx):
    item = self.dataset[idx]
    return{
        "prompt": item["prompt"],
        "chosen": item["chosen"],
        "rejected": item["rejected"]
    }

In [5]:
model_name = "distilgpt2"
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token

model = AutoModelForCausalLM.from_pretrained(model_name)
ref_model = AutoModelForCausalLM.from_pretrained(model_name)

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/762 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/353M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

In [13]:
class DPOTrainer:
  def __init__(self,model,ref_model,tokenizer,beta=0.1,lr=1e-5,device="cuda"):
    self.model=model.to(device)
    self.ref_model = ref_model.to(device)
    self.tokenizer = tokenizer
    self.beta = beta
    self.device = device
    self.optimizer = Adam(self.model.parameters(),lr=lr)

    for p in self.ref_model.parameters():
      p.requires_grad = False
    self.ref_model.eval()

  def prepare_inputs(self,prompts,responses):
    texts = [p + r for p,r in zip(prompts,responses)]
    inputs = self.tokenizer(texts,return_tensors="pt",padding=True,truncation=True).to(self.device)
    return inputs

  def compute_log_probs(self,model,inputs):
    input_ids = inputs["input_ids"]
    attention_mask = inputs["attention_mask"]

    outputs = model(input_ids=input_ids,attention_mask=attention_mask)
    logits = outputs.logits[:,:-1,:]
    log_probs = torch.nn.functional.log_softmax(logits,dim=-1)

    target_ids = input_ids[:,1:]
    mask = attention_mask[:,1:].bool()
    selected_log_probs = log_probs.gather(2,target_ids.unsqueeze(-1)).squeeze(-1)
    selected_log_probs = selected_log_probs * mask

    return selected_log_probs.sum(dim=1)

  def dpo_loss(self,prompts,chosen,rejected):
    inputs_chosen = self.prepare_inputs(prompts,chosen)
    inputs_rejected = self.prepare_inputs(prompts,rejected)

    logp_chosen = self.compute_log_probs(self.model,inputs_chosen)
    logp_rejected = self.compute_log_probs(self.model,inputs_rejected)

    with torch.no_grad():
      logp_ref_chosen = self.compute_log_probs(self.ref_model,inputs_chosen)
      logp_ref_rejected = self.compute_log_probs(self.ref_model,inputs_rejected)

    diff_model = logp_chosen - logp_rejected
    diff_ref = logp_ref_chosen - logp_ref_rejected

    loss = -torch.nn.functional.logsigmoid(self.beta * (diff_model - diff_ref)).mean()
    return loss

  def train(self,dataset,epochs=3,batch_size=4):
    dataloader = DataLoader(dataset,batch_size=batch_size,shuffle=True)

    for epoch in range(epochs):
      total_loss = 0
      for batch in tqdm(dataloader,desc=f"Epoch {epoch+1}"):
        prompt = batch["prompt"]
        chosen = batch["chosen"]
        rejected = batch["rejected"]

        loss = self.dpo_loss(prompt,chosen,rejected)

        self.optimizer.zero_grad()
        loss.backward()
        self.optimizer.step()

        total_loss += loss.item()

      avg_loss = total_loss / len(dataloader)
      print(f"Epoch {epoch+1} - Loss: {avg_loss:.4f}")


In [14]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
trainer = DPOTrainer(model,ref_model,tokenizer,device=device)
trainer.train(dataset,epochs=3,batch_size=2)

Epoch 1: 100%|██████████| 1657/1657 [05:11<00:00,  5.33it/s]


Epoch 1 - Loss: 0.0974


Epoch 2: 100%|██████████| 1657/1657 [05:03<00:00,  5.47it/s]


Epoch 2 - Loss: 0.0184


Epoch 3: 100%|██████████| 1657/1657 [05:03<00:00,  5.47it/s]

Epoch 3 - Loss: 0.0027
